In [3]:
from elasticsearch import Elasticsearch
import csv

In [9]:
es = Elasticsearch(hosts=[{'host': '40.114.92.165', 'port': '9210'}],http_auth=('elastic', 'elastic@123'))

In [10]:
data = es.search(index='live_site_data', body={
    "size": 400,
    "query": {
        "match_all": {}
    }
})

In [11]:
datas = data['hits']['hits']

In [12]:
datas[0]

{'_index': 'live_site_data',
 '_type': 'cbuilder_row',
 '_id': 'WULhPHcB9z14MoqeHHUV',
 '_score': 1.0,
 '_source': {'type': 'row',
  'properties': {'backgroundImage': False},
  'components': ['column', 'heading', 'text'],
  'count': [{'heading': 1}, {'text': 1}, {'image': 0}],
  'componentCount': 3,
  'nodes': ['column'],
  'child': [{'type': 'column',
    'properties': {'backgroundImage': False},
    'components': ['heading', 'text'],
    'nodes': ['heading', 'text'],
    'child': [{'type': 'heading',
      'properties': {'backgroundImage': False},
      'components': [],
      'nodes': [],
      'child': []},
     {'type': 'text',
      'properties': {'backgroundImage': False},
      'components': [],
      'nodes': [],
      'child': []}]}],
  'template': '<div class="cRow TxAl-c Pt-90 Pb-40" data-id="656" data-type="row"><div class="editor-row-container container-medium"><div class="cGrid "><div class="editor-col cCol ui-state-default sfFixed  mMt-0 mMb-0 mTxAl-n mTxAl-o mDib tDib 

In [5]:
all_keys = []
for data in datas:
    keys = set().union(*(d.keys() for d in data['_source']['count']))
    all_keys.extend([k.lower() for k in keys])

all_keys = list(set(all_keys))
all_keys.sort()

In [7]:
with open('dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    top_row = ['Id', 'Name']
    top_row.extend(all_keys)
    writer.writerow(top_row)
    for data in datas:
        row_data = [data['_id'], 'PlaceHolderName']
        count = data['_source']['count']
        for key in all_keys:
            if key in data['_source']['components']:
                row_data.extend([sub[key] for sub in count if key in sub.keys()])
            else:
                row_data.append(0)
        writer.writerow(row_data)